# Double Joint

A double joint test notebook. Included are the algorithm and interactive example.

In [ ]:
# First, let's setup the notebook
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets

## Step-by-step Algorithm

### 1. Create rotation matrix

A rotation matrix with axis
$\begin{bmatrix} \sin(rx) \\ 0 \\ \cos(rx) \end{bmatrix}$
and angle $ry$ is:

$
\mathbf{R} = \begin{bmatrix}
\cos(ry) + \sin(rx)^2*(1 - \cos(ry)) & -\cos(rx)*\sin(ry) & \sin(rx)*\cos(rx)*(1 - \cos(ry)) \\
\cos(rx)*\sin(ry) & \cos(ry) & -\sin(rx)*\sin(ry) \\
\sin(rx)*\cos(rx)*(1 - \cos(ry)) & \sin(rx)*\sin(ry) & \cos(ry) + \cos(rx)^2*(1 - \cos(ry)) \\
\end{bmatrix}
$

[Modified from Wikipedia](https://en.wikipedia.org/wiki/Rotation_matrix)

In [ ]:
def create_rotation_matrix(rx, ry):
    srx, crx = np.sin(rx), np.cos(rx)
    sry, cry = np.sin(ry), np.cos(ry)
    return np.matrix([[cry + srx**2*(1 - cry), -crx*sry, srx*crx*(1 - cry)],
                      [crx*sry, cry, -srx*sry],
                      [srx*crx*(1 - cry), srx*sry, cry + crx**2*(1 - cry)]])

### 2. Find point in rotated xy-plane

Basically, we want to find y that satisfies the following equation:

$\mathbf{R} \begin{bmatrix} x_0 \\ 0 \\ z_0 \end{bmatrix} = \begin{bmatrix} x \\ y \\ z \end{bmatrix}$

Step by step:
1. Define $x$ and $z$

2. Find $x_0$ and $z_0$

 To find it, we can solve the following equation:
 $
 \begin{bmatrix} \mathbf{r}_{11} & \mathbf{r}_{13} \\ \mathbf{r}_{31} & \mathbf{r}_{33} \end{bmatrix} \begin{bmatrix} x_0 \\ z_0 \end{bmatrix} = \begin{bmatrix} x \\ z \end{bmatrix}
 $

3. $y = \mathbf{r}_{21} * x_0 + \mathbf{r}_{23} * z_0$

In [ ]:
def find_point_in_rotated_plane(R, x, z):
    # find x_0 and z_0
    det_m = (R[0,0]*R[2,2]-R[2,0]*R[0,2])
    x_0, z_0 = R[2,2]*x-R[0,2]*z, -R[2,0]*x+R[0,0]*z
    x_0, z_0 = x_0/det_m, z_0/det_m
    
    # Calculate and return Y
    return R[1,0]*x_0 + R[1,2]*z_0

### 3. Create bilinear gradient map

Now that we find the y-coordinate of the corners, now we can map _every_ point within our original plane to transformed plane.

In [ ]:
def bilinear_gradient(x_00, x_01, x_10, x_11, x_size, y_size):
    # Create the corner matrix
    x_arr, y_arr = np.linspace(1.0, 0.0, x_size), np.linspace(1.0, 0.0, y_size)
    mat = np.multiply.outer(x_arr, y_arr)
    
    # Allocate return and temporary matrix
    temp = np.empty_like(mat)
    ret = np.zeros_like(mat)
    
    # Multiply-and-accumulate the matrix with each corner
    np.add(np.multiply(mat, x_00, out=temp), ret, out=ret)
    np.add(np.multiply(mat[:, ::-1], x_01, out=temp), ret, out=ret)
    np.add(np.multiply(mat[::-1, :], x_10, out=temp), ret, out=ret)
    np.add(np.multiply(mat[::-1, ::-1], x_11, out=temp), ret, out=ret)
    
    return ret

### 4. _Warp_ the arm

Multiply each y-coordinate of our arm with the transformed plane.

In [ ]:
def create_arm(size):
    # Create half the arm
    x_arr = z_arr = np.linspace(-1.0, 1.0, size+1)
    y_len = (size+1)//2
    y_arr = np.linspace(0, y_len, y_len+1, dtype=np.float64)
    if size % 2 == 1:
        y_arr[-1] -= 0.5 # If odd, one voxel must be half size
    y_arr *= 0.5 / y_arr[-1] # Make them fit in a 1x0.5x1 box
    
    ret = np.empty((x_arr.size, y_arr.size, z_arr.size, 3), dtype=np.float64)
    
    ret[..., 0] = x_arr[:, np.newaxis, np.newaxis]
    ret[..., 1] = y_arr[np.newaxis, :, np.newaxis]
    ret[..., 2] = z_arr[np.newaxis, np.newaxis, :]
    
    return ret

def warp_arm(arm, trans_plane):
    arm[..., 1] *= trans_plane[:, np.newaxis, :]

### 5. Clone the arm and rotate

To make the other side of the arm, clone and rotate the arm with our rotation matrix

In [ ]:
def flip_arm(arm, R):
    clone = np.copy(arm)
    mat = R @ R @ np.diag([1, -1, 1]) # flip the clone upside down first, then rotate twice
    clone = clone[..., np.newaxis] # Turn clone into an array of vectors (last dimension is 1)
    np.matmul(mat, clone, out=clone)
    return clone[..., 0] # "unvectorize" and return

## Combining them all

Now for the example program

In [ ]:
@widgets.interact(rx=widgets.BoundedFloatText(value=0, min=-180, max=180, step=0.1, description='Rotation X'),
                  ry=widgets.BoundedFloatText(value=90, min=-90, max=90, step=0.1, description='Rotation Y'),
                  size=widgets.IntSlider(value=2, min=1, max=10, description='Size'),
                  faces=widgets.Checkbox(True, description='Fill'))
def example_arm(rx, ry, size, faces):
    rx, ry = np.radians(rx), np.radians(ry)
    
    R = create_rotation_matrix(rx, ry/2)
    
    y_00 = find_point_in_rotated_plane(R, -1, -1)
    y_01 = find_point_in_rotated_plane(R, -1, 1)
    y_11 = find_point_in_rotated_plane(R, 1, 1)
    y_10 = find_point_in_rotated_plane(R, 1, -1)
    
    g = bilinear_gradient(y_00, y_01, y_10, y_11, size+1, size+1)
    # Normalize g
    g += np.sqrt(2)
    g *= 2
    
    arm = create_arm(size)
    warp_arm(arm, g)
    arm[..., 1] -= np.sqrt(2) # Make sure center of the arm is at origin
    
    otherside = flip_arm(arm[:, :-1], R) # Do not include potentially duplicate points
    arm = np.concatenate((arm, otherside[:, ::-1]), axis=1) # Combine them all
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.view_init(45, 45)
    lim = (-np.sqrt(2), np.sqrt(2))
    ax.set_xlim(*lim)
    ax.set_ylim(*lim)
    ax.set_zlim(*lim)
    
    X, Y, Z = arm[..., 0], arm[..., 1], arm[..., 2]
    larm_len = (size+1)//2
    edges = np.empty((size, arm.shape[1]-1, size, 3), dtype=np.float64)
    edges[:, :larm_len, :] = (1, 0, 0)
    edges[:, larm_len:, :] = (0, 0, 1)
    if size % 2 == 1:
        edges[:, larm_len-1, :] = (1, 0, 0.5)
        edges[:, larm_len, :] = (0.5, 0, 1)
    if faces:
        facecolors = np.empty(edges.shape[:-1] + (4,))
        facecolors[..., :3] = edges
        facecolors[..., 3] = 0.5
    else:
        facecolors = (0, 0, 0, 0)
    ax.voxels(X, Z, Y,
              np.broadcast_to(True, np.array(arm.shape[:-1])-1),
              facecolors=facecolors,
              edgecolors=edges,
              shade=True,
              lightsource=colors.LightSource(azdeg=180, altdeg=45),
             )

## Steve Renderer

Now to the real deal, **STEVE!!!**

This example below use double joint to render steve

In [ ]:
from PIL import Image

class BloxDrawer:
    def __init__(self,
                 p000=(0.0, 0.0, 0.0),
                 p001=(0.0, 0.0, 1.0),
                 p010=(0.0, 1.0, 0.0),
                 p011=(0.0, 1.0, 1.0),
                 p100=(1.0, 0.0, 0.0),
                 p101=(1.0, 0.0, 1.0),
                 p110=(1.0, 1.0, 0.0),
                 p111=(1.0, 1.0, 1.0),
                 *,
                 face_up=None,
                 face_down=None,
                 face_left=None,
                 face_right=None,
                 face_front=None,
                 face_back=None,
                 color=None,
                ):
        
        if color is not None:
            if (face_up is not None or
                face_down is not None or
                face_left is not None or
                face_right is not None or
                face_front is not None or
                face_back is not None
               ):
                raise ValueError('if color is supplied, no face shall be specified')
            face_up = face_down = face_left = face_right = face_front = face_back = color
        self.face_up = face_up
        self.face_down = face_down
        self.face_left = face_left
        self.face_right = face_right
        self.face_front = face_front
        self.face_back = face_back
        
        points = np.array([p000, p001,
                           p010, p011,
                           p100, p101,
                           p110, p111,
                          ], dtype=np.float64).reshape((2, 2, 2, 3))
        points = np.concatenate((points, np.broadcast_to(1.0, (2, 2, 2, 1))),
                                axis=-1,
                               )
        self._points = points
    
    @property
    def points(self):
        return self._points[..., :3]
    
    def transform(self, mat):
        np.matmul(mat, self._points[..., np.newaxis], out=self._points[..., np.newaxis])
        return self
    
    def draw(self, ax, **kwarg):
        ix_ = np.ix_([0, 1], [0, 1])
        if 'edgecolor' in kwarg:
            edgecolor = kwarg.pop('edgecolor')
        elif 'edgecolors' in kwarg:
            edgecolor = kwarg.pop('edgecolors')
        else:
            edgecolor = None
        self._draw_face(self.points[0, :, :][ix_], self.face_left, ax, kwarg, edgecolor=edgecolor)
        self._draw_face(self.points[:, 0, :][ix_], self.face_down, ax, kwarg, edgecolor=edgecolor)
        self._draw_face(self.points[:, :, 0][ix_], self.face_back, ax, kwarg, edgecolor=edgecolor)
        self._draw_face(self.points[1, :, :][ix_], self.face_right, ax, kwarg, edgecolor=edgecolor)
        self._draw_face(self.points[:, 1, :][ix_], self.face_up, ax, kwarg, edgecolor=edgecolor)
        self._draw_face(self.points[:, :, 1][ix_], self.face_front, ax, kwarg, edgecolor=edgecolor)
    
    def _draw_face(self, points, face, ax, kw, edgecolor=None):
        if face is None:
            return # No face is drawn
        if np.ndim(face) in (0, 1):
            face = np.reshape(face, (1, 1, -1))
        elif np.ndim(face) not in (2, 3):
            raise ValueError('incorrect dimension for face')
        x, y = np.shape(face)[:2]
        x, y = x+1, y+1
        px = bilinear_gradient(points[0, 0, 0], points[0, 1, 0],
                               points[1, 0, 0], points[1, 1, 0],
                               x, y)
        py = bilinear_gradient(points[0, 0, 1], points[0, 1, 1],
                               points[1, 0, 1], points[1, 1, 1],
                               x, y)
        pz = bilinear_gradient(points[0, 0, 2], points[0, 1, 2],
                               points[1, 0, 2], points[1, 1, 2],
                               x, y)
        tri = ax.plot_surface(px, pz, py,
                              facecolors=face,
                              **kw,
                             )
        if edgecolor is not None:
            tri.set_edgecolor(edgecolor)

def move_mat(dx, dy, dz):
    ret = np.mat(np.eye(4))
    ret[:3, -1] = np.array([dx, dy, dz])[:, np.newaxis]
    return ret

def scale_mat(sx, sy, sz):
    return np.diag([sx, sy, sz, 1])

def swap_mat(ax1, ax2):
    ret = np.mat(np.eye(4))
    ret[np.ix_([ax1, ax2], [ax1, ax2])] = np.mat([[0, 1],
                                                  [1, 0]])
    return ret

def rot_mat(axis, angle):
    ca, sa = np.cos(angle), np.sin(angle)
    mat = np.mat([[ca, -sa],
                  [sa, ca]])
    ret = np.mat(np.eye(4))
    if axis == 0:
        ix = np.ix_([1, 2], [1, 2])
    elif axis == 1:
        ix = np.ix_([0, 2], [0, 2])
    elif axis == 2:
        ix = np.ix_([0, 1], [0, 1])
    else:
        raise ValueError('axis not recognized')
    ret[ix] = mat
    return ret

class DoubleJointDrawer:
    def __init__(self, *,
                 face_up=None,
                 face_down=None,
                 face_left=None,
                 face_right=None,
                 face_front=None,
                 face_back=None,
                 color=None,
                 **kw,
                ):
        def split(face, splitx=False):
            if face is None:
                return None, None
            elif np.ndim(face) in (0, 1):
                return face, face
            else:
                face = np.array(face, copy=False)
                x, y = face.shape[:2]
                if splitx:
                    return face[:(x+1)//2, :], face[x//2:, :]
                else:
                    return face[:, :(y+1)//2], face[:, y//2:]
        if color is None:
            lb, lu = split(face_left, splitx=True)
            rb, ru = split(face_right, splitx=True)
            fb, fu = split(face_front)
            bb, bu = split(face_back)
            bloxb = BloxDrawer(face_up=None,
                               face_down=face_down,
                               face_left=lb,
                               face_right=rb,
                               face_front=fb,
                               face_back=bb,
                               **kw,
                              )
            bloxu = BloxDrawer(face_up=face_up,
                               face_down=None,
                               face_left=lu,
                               face_right=ru,
                               face_front=fu,
                               face_back=bu,
                               **kw,
                              )
        else:
            if (face_up is not None or
                face_down is not None or
                face_left is not None or
                face_right is not None or
                face_front is not None or
                face_back is not None
               ):
                raise ValueError('if color is supplied, no face shall be specified')
            bloxb = BloxDrawer(face_up=None,
                               face_down=color,
                               face_left=color,
                               face_right=color,
                               face_front=color,
                               face_back=color,
                               **kw,
                              )
            bloxu = BloxDrawer(face_up=color,
                               face_down=None,
                               face_left=color,
                               face_right=color,
                               face_front=color,
                               face_back=color,
                               **kw,
                              )
        
        self._bloxb, self._bloxu = bloxb, bloxu
    
    __sqrt_2 = np.sqrt(0.5)
    
    __defaultpoint = np.stack(np.meshgrid([-0.5, 0.5], [-__sqrt_2, 0, __sqrt_2], [-0.5, 0.5], indexing='ij'), axis=-1)
    
    def generate(self, rx, ry):
        
        self._bloxb.points[...] = self.__defaultpoint[:, :2, :]
        self._bloxu.points[...] = self.__defaultpoint[:, 1:, :]
        
        R = create_rotation_matrix(rx, ry/2)

        y_00 = find_point_in_rotated_plane(R, -1, -1)
        y_01 = find_point_in_rotated_plane(R, -1, 1)
        y_11 = find_point_in_rotated_plane(R, 1, 1)
        y_10 = find_point_in_rotated_plane(R, 1, -1)

        g = np.array([[y_00, y_01],
                      [y_10, y_11]])
        g /= 2

        self._bloxb.points[:, 1, :, 1] = g
        self._bloxu.points[:, 0, :, 1] = g
        
        r4 = np.eye(4)
        r4[:3, :3] = R @ R
        
        np.matmul(r4[:3, :3], self._bloxu._points[:, 1, :, :3, np.newaxis], out=self._bloxu._points[:, 1, :, :3, np.newaxis])
        
        self._bloxb.points[..., 1] += self.__sqrt_2
        self._bloxu.points[..., 1] += self.__sqrt_2
        
        return move_mat(0, self.__sqrt_2, 0) @ r4 @ move_mat(-0.5, self.__sqrt_2, -0.5)
    
    def transform(self, ma):
        self._bloxb.transform(ma)
        self._bloxu.transform(ma)
        return self
    
    def draw(self, *a, **kw):
        self._bloxb.draw(*a, **kw)
        self._bloxu.draw(*a, **kw)

class SingleJointDrawer(DoubleJointDrawer):
    
    __defaultpoint = np.stack(np.meshgrid([-0.5, 0.5], [-0.5, 0, 0.5], [-0.5, 0.5], indexing='ij'), axis=-1)
    
    def generate(self, r):
        
        self._bloxb.points[...] = self.__defaultpoint[:, :2, :]
        self._bloxu.points[...] = self.__defaultpoint[:, 1:, :]
        
        sr, cr, tr2_2 = np.sin(r), np.cos(r), np.tan(r / 2) / 2
        R = np.mat([[cr, sr],
                    [-sr, cr]])
        
        self._bloxb.points[0, 1, :, 1] = tr2_2
        self._bloxu.points[0, 0, :, 1] = tr2_2
        self._bloxb.points[1, 1, :, 1] = -tr2_2
        self._bloxu.points[1, 0, :, 1] = -tr2_2
        np.matmul(R, self._bloxu.points[:, 1, :, :2, np.newaxis], out=self._bloxu.points[:, 1, :, :2, np.newaxis])
        
        r4 = np.eye(4)
        r4[:2, :2] = R
        
        self._bloxb.points[..., 1] += 0.5
        self._bloxu.points[..., 1] += 0.5
        
        return move_mat(0, 0.5, 0) @ r4 @ move_mat(-0.5, 0.5, -0.5)

arrow = np.array([[0, 0, 1, 0, 0],
                  [0, 1, 1, 1, 0],
                  [1, 1, 1, 1, 1],
                  [0, 0, 1, 0, 0],
                  [0, 0, 1, 0, 0],
                 ])
arrow = np.array([[1, 1, 1],
                  [0, 0, 0],
                 ], dtype=np.float64)[arrow]

@widgets.interact(
    rax=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Right Shoulder Angle (X)'),
    ray=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Right Shoulder Angle (Y)'),
    raz=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Right Shoulder Angle (Z)'),
    ra=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Right Elbow Angle'),
    lax=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Left Shoulder Angle (X)'),
    lay=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Left Shoulder Angle (Y)'),
    laz=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Left Shoulder Angle (Z)'),
    la=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Left Elbow Angle'),
    rlx=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Right Leg Angle (X)'),
    rly=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Right Leg Angle (Y)'),
    rlz=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Right Leg Angle (Z)'),
    rl=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Right Knee Angle'),
    llx=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Left Leg Angle (X)'),
    lly=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Left Leg Angle (Y)'),
    llz=widgets.FloatSlider(value=0, min=-180, max=180, step=0.1, description='Left Leg Angle (Z)'),
    ll=widgets.FloatSlider(value=0, min=-90, max=90, step=0.1, description='Left Knee Angle'),
)
def draw_steve(rax, ray, raz, ra, lax, lay, laz, la, rlx, rly, rlz, rl, llx, lly, llz, ll):
    rax, ray, raz, ra, lax, lay, laz, la, rlx, rly, rlz, rl, llx, lly, llz, ll = np.radians([rax, ray, raz, ra, lax, lay, laz, la, rlx, rly, rlz, rl, llx, lly, llz, ll])
    
    # Create frame
    fig = plt.figure()
    ax = fig.gca(projection='3d', proj_type='ortho')
    ax.view_init(45, 45)
    ax.set_xlabel('x')
    ax.set_ylabel('z')
    ax.set_zlabel('y')
    ax.set_xlim(-16, 16)
    ax.set_ylim(-16, 16)
    ax.set_zlim(-22, 10)
    
    im = Image.open('test_skin.png')
    def get_im(left, upper, right, bottom):
        ret = np.array(im.crop([left, upper, right, bottom]), dtype=np.float64)[..., :3]
        ret /= 255
        return ret
    
    kw = {
        'shade': True,
        'lightsource': colors.LightSource(azdeg=90, altdeg=-45),
    }
    
    # Draw body
    body = BloxDrawer(
        face_up=get_im(20, 16, 28, 20).swapaxes(0, 1)[::-1, :],
        face_down=get_im(28, 16, 36, 20).swapaxes(0, 1)[::-1, :],
        face_left=get_im(28, 20, 32, 32)[::-1, ::-1],
        face_right=get_im(16, 20, 20, 32)[::-1, :],
        face_front=get_im(20, 20, 28, 32).swapaxes(0, 1)[:, ::-1],
        face_back=get_im(32, 20, 40, 32).swapaxes(0, 1)[:, ::-1],
    ).transform(move_mat(-4, -10, -2) @ scale_mat(8, 12, 4))
    body.draw(ax, **kw)
    
    # Draw head
    head = BloxDrawer(
        face_up=get_im(8, 0, 16, 8).swapaxes(0, 1)[::-1, :],
        face_down=get_im(16, 0, 24, 8).swapaxes(0, 1)[::-1, :],
        face_left=get_im(16, 8, 24, 16)[::-1, ::-1],
        face_right=get_im(0, 8, 8, 16)[::-1, :],
        face_front=get_im(8, 8, 16, 16).swapaxes(0, 1)[:, ::-1],
        face_back=get_im(24, 8, 32, 16).swapaxes(0, 1)[:, ::-1],
    ).transform(move_mat(-4, 2, -4) @ scale_mat(8, 8, 8))
    head.draw(ax, **kw)
    
    # Draw right shoulder
    rshoulder = DoubleJointDrawer(
        face_down=get_im(44, 16, 48, 20).swapaxes(0, 1)[::-1, :],
        face_left=get_im(48, 20, 52, 24)[:, ::-1],
        face_right=get_im(40, 20, 44, 24),
        face_front=get_im(44, 20, 48, 24).swapaxes(0, 1)[::-1, :],
        face_back=get_im(52, 20, 56, 24).swapaxes(0, 1),
    )
    t2 = move_mat(-4, 0, 0) @ scale_mat(-4, 4, 4) @ rot_mat(0, raz) @ swap_mat(1, 0)
    t1 = t2 @ rshoulder.generate(rax, ray) @ scale_mat(0.25, 0.25, 0.25)
    rshoulder.transform(t2)
    rshoulder.draw(ax, **kw)
    
    # Draw right elbow
    relbow = SingleJointDrawer(
        face_left=get_im(48, 24, 52, 28)[:, ::-1],
        face_right=get_im(40, 24, 44, 28),
        face_front=get_im(44, 24, 48, 28).swapaxes(0, 1)[::-1, :],
        face_back=get_im(52, 24, 56, 28).swapaxes(0, 1),
    )
    t2 = t1 @ scale_mat(4, 4, 4) @ move_mat(0.5, 0, 0.5)
    t1 = t2 @ relbow.generate(ra) @ scale_mat(0.25, 0.25, 0.25)
    relbow.transform(t2)
    relbow.draw(ax, **kw)
    
    # Draw right arm
    rarm = BloxDrawer(
        face_up=get_im(48, 16, 52, 20).swapaxes(0, 1),
        face_left=get_im(48, 28, 52, 32)[:, ::-1],
        face_right=get_im(40, 28, 44, 32),
        face_front=get_im(44, 28, 48, 32).swapaxes(0, 1)[::-1, :],
        face_back=get_im(52, 28, 56, 32).swapaxes(0, 1),
    ).transform(t1 @ scale_mat(4, 4, 4))
    rarm.draw(ax, **kw)
    
    # Draw left shoulder
    lshoulder = DoubleJointDrawer(
        face_down=get_im(36, 48, 40, 52).swapaxes(0, 1),
        face_left=get_im(32, 52, 36, 56),
        face_right=get_im(40, 52, 44, 56)[:, ::-1],
        face_front=get_im(36, 52, 40, 56).swapaxes(0, 1),
        face_back=get_im(44, 52, 48, 56).swapaxes(0, 1)[::-1, :],
    )
    t2 = move_mat(4, 0, 0) @ scale_mat(4, 4, 4) @ rot_mat(0, -laz) @ swap_mat(1, 0)
    t1 = t2 @ lshoulder.generate(lax, lay) @ scale_mat(0.25, 0.25, 0.25)
    lshoulder.transform(t2)
    lshoulder.draw(ax, **kw)
    
    # Draw left elbow
    lelbow = SingleJointDrawer(
        face_left=get_im(32, 56, 36, 60),
        face_right=get_im(40, 56, 44, 60)[:, ::-1],
        face_front=get_im(36, 56, 40, 60).swapaxes(0, 1),
        face_back=get_im(44, 56, 48, 60).swapaxes(0, 1)[::-1, :],
    )
    t2 = t1 @ scale_mat(4, 4, 4) @ move_mat(0.5, 0, 0.5)
    t1 = t2 @ lelbow.generate(la) @ scale_mat(0.25, 0.25, 0.25)
    lelbow.transform(t2)
    lelbow.draw(ax, **kw)
    
    # Draw left arm
    larm = BloxDrawer(
        face_up=get_im(40, 48, 44, 52).swapaxes(0, 1)[::-1, :],
        face_left=get_im(32, 60, 36, 64),
        face_right=get_im(40, 60, 44, 64)[:, ::-1],
        face_front=get_im(36, 60, 40, 64).swapaxes(0, 1),
        face_back=get_im(44, 60, 48, 64).swapaxes(0, 1)[::-1, :],
    ).transform(t1 @ scale_mat(4, 4, 4))
    larm.draw(ax, **kw)
    
    # Draw right leg
    rleg1 = DoubleJointDrawer(
        face_down=get_im(4, 16, 8, 20)[::-1, :],
        face_left=get_im(4, 20, 8, 24),
        face_right=get_im(12, 20, 16, 24)[:, ::-1],
        face_front=get_im(0, 20, 4, 24).swapaxes(0, 1)[::-1, :],
        face_back=get_im(8, 20, 12, 24).swapaxes(0, 1),
    )
    t2 = move_mat(-2, -10, 0) @ scale_mat(-4, -4, -4)  @ rot_mat(1, rlz) @ swap_mat(0, 2)
    t1 = t2 @ rleg1.generate(rlx, rly) @ scale_mat(0.25, 0.25, 0.25)
    rleg1.transform(t2)
    rleg1.draw(ax, **kw)
    
    rleg2 = SingleJointDrawer(
        face_left=get_im(4, 24, 8, 28),
        face_right=get_im(12, 24, 16, 28)[:, ::-1],
        face_front=get_im(0, 24, 4, 28).swapaxes(0, 1)[::-1, :],
        face_back=get_im(8, 24, 12, 28).swapaxes(0, 1),
    )
    t2 = t1 @ scale_mat(4, 4, 4) @ move_mat(0.5, 0, 0.5)
    t1 = t2 @ rleg2.generate(rl) @ scale_mat(0.25, 0.25, 0.25)
    rleg2.transform(t2)
    rleg2.draw(ax, **kw)
    
    rleg3 = BloxDrawer(
        face_up=get_im(8, 16, 12, 20)[::-1, :],
        face_left=get_im(4, 28, 8, 32),
        face_right=get_im(12, 28, 16, 32)[:, ::-1],
        face_front=get_im(0, 28, 4, 32).swapaxes(0, 1)[::-1, :],
        face_back=get_im(8, 28, 12, 32).swapaxes(0, 1),
    ).transform(t1 @ scale_mat(4, 4, 4))
    rleg3.draw(ax, **kw)
    
    # Draw left leg
    lleg1 = DoubleJointDrawer(
        face_down=get_im(20, 48, 24, 52)[::-1, ::-1],
        face_left=get_im(20, 52, 24, 56)[:, ::-1],
        face_right=get_im(28, 52, 32, 56),
        face_front=get_im(24, 52, 28, 56).swapaxes(0, 1),
        face_back=get_im(16, 52, 20, 56).swapaxes(0, 1)[::-1, :],
    )
    t2 = move_mat(2, -10, 0) @ scale_mat(4, -4, -4)  @ rot_mat(1, -llz) @ swap_mat(0, 2)
    t1 = t2 @ lleg1.generate(llx, lly) @ scale_mat(0.25, 0.25, 0.25)
    lleg1.transform(t2)
    lleg1.draw(ax, **kw)
    
    lleg2 = SingleJointDrawer(
        face_left=get_im(20, 56, 24, 60)[:, ::-1],
        face_right=get_im(28, 56, 32, 60),
        face_front=get_im(24, 56, 28, 60).swapaxes(0, 1),
        face_back=get_im(16, 56, 20, 60).swapaxes(0, 1)[::-1, :],
    )
    t2 = t1 @ scale_mat(4, 4, 4) @ move_mat(0.5, 0, 0.5)
    t1 = t2 @ lleg2.generate(ll) @ scale_mat(0.25, 0.25, 0.25)
    lleg2.transform(t2)
    lleg2.draw(ax, **kw)
    
    lleg3 = BloxDrawer(
        face_up=get_im(24, 48, 28, 52)[::-1, ::-1],
        face_left=get_im(20, 60, 24, 64)[:, ::-1],
        face_right=get_im(28, 60, 32, 64),
        face_front=get_im(24, 60, 28, 64).swapaxes(0, 1),
        face_back=get_im(16, 60, 20, 64).swapaxes(0, 1)[::-1, :],
    ).transform(t1 @ scale_mat(4, 4, 4))
    lleg3.draw(ax, **kw)
    
    im.close()
